In [67]:
import pandas as pd
import numpy as np
from sklearn import linear_model
import matplotlib.pyplot as plt

In [68]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [69]:
data = pd.read_csv("candida.csv")

In [70]:
data

,example,eyes,nose,head,fcolor,hair,smile
0,1,round,triangle,round,purple,yes,yes
1,2,squre,squre,squre,yellow,yes,no
2,3,squre,trianle,squre,green,yes,yes
3,4,round,trianle,squre,yellow,no,no
4,5,squre,squre,squre,green,yes,yes


In [71]:
import random
import csv

In [72]:
def more_general(h1, h2):
    more_general_parts = []
def g_0(n):
    return("?")*n
def s_0(n):
    return("0")*n
    for x, y in zip(h1, h2):
        mg = x == "?" or (x != "0" and (x == y or y == "0"))
        more_general_parts.append(mg)
    return all(more_general_parts)

l1 = [1, 2, 3]
l2 = [3, 4, 5]

list(zip(l1, l2))

[(1, 3), (2, 4), (3, 5)]

In [73]:
# min_generalizations
def fulfills(example, hypothesis):
    ### the implementation is the same as for hypotheses:
    return more_general(hypothesis, example)

def min_generalizations(h, x):
    h_new = list(h)
    for i in range(len(h)):
        if not fulfills(x[i:i+1], h[i:i+1]):
            h_new[i] = '?' if h[i] != '0' else x[i]
    return [tuple(h_new)]

In [74]:
min_generalizations(h=('0', '0'  , 'sunny'), 
                    x=('eyes', 'nose', 'head','fcolor'))

[('eyes', 'nose', '?')]

In [75]:
def min_specializations(h, domains, x):
    results = []
    for i in range(len(h)):
        if h[i] == "?":
            for val in domains[i]:
                if x[i] != val:
                    h_new = h[:i] + (val,) + h[i+1:]
                    results.append(h_new)
        elif h[i] != "0":
            h_new = h[:i] + ('0',) + h[i+1:]
            results.append(h_new)
    return results

In [76]:
min_specializations(h=('?', 'x',), 
                    domains=[['a', 'b', 'c'], ['x', 'y']], 
                    x=('b', 'x'))

[('a', 'x'), ('c', 'x'), ('?', '0')]

In [77]:
with open('candida.csv')  as csvFile:
        examples = [tuple(line) for line in csv.reader(csvFile)]
        

In [78]:
  examples

[('example', 'eyes', 'nose', 'head', 'fcolor', 'hair', 'smile'),
 ('1', 'round', 'triangle', 'round', 'purple', 'yes', 'yes'),
 ('2', 'squre', 'squre', 'squre', 'yellow', 'yes', 'no'),
 ('3', 'squre', 'trianle', 'squre', 'green', 'yes', 'yes'),
 ('4', 'round', 'trianle', 'squre', 'yellow', 'no', 'no'),
 ('5', 'squre', 'squre', 'squre', 'green', 'yes', 'yes')]

In [79]:
def get_domains(examples):
    d = [set() for i in examples[0]]
    for x in examples:
        for i, xi in enumerate(x):
            d[i].add(xi)
    return [list(sorted(x)) for x in d]

get_domains(examples)

[['1', '2', '3', '4', '5', 'example'],
 ['eyes', 'round', 'squre'],
 ['nose', 'squre', 'triangle', 'trianle'],
 ['head', 'round', 'squre'],
 ['fcolor', 'green', 'purple', 'yellow'],
 ['hair', 'no', 'yes'],
 ['no', 'smile', 'yes']]

In [80]:
def candidate_elimination(examples):
    domains = get_domains(examples)[:-1]
    
    G = set([g_0(len(domains))])
    S = set([s_0(len(domains))])
    i=0
    print("\n G[{0}]:".format(i),G)
    print("\n S[{0}]:".format(i),S)
    for xcx in examples:
        i=i+1
        x, cx = xcx[:-1], xcx[-1]  # Splitting data into attributes and decisions
        if cx=='Y': # x is positive example
            G = {g for g in G if fulfills(x, g)}
            S = generalize_S(x, G, S)
        else: # x is negative example
            S = {s for s in S if not fulfills(x, s)}
            G = specialize_G(x, domains, G, S)
        print("\n G[{0}]:".format(i),G)
        print("\n S[{0}]:".format(i),S)
    return


In [81]:
def generalize_S(x, G, S):
    S_prev = list(S)
    for s in S_prev:
        if s not in S:
            continue
        if not fulfills(x, s):
            S.remove(s)
            Splus = min_generalizations(s, x)
            ## keep only generalizations that have a counterpart in G
            S.update([h for h in Splus if any([more_general(g,h) 
                                               for g in G])])
            ## remove hypotheses less specific than any other in S
            S.difference_update([h for h in S if 
                                 any([more_general(h, h1) 
                                      for h1 in S if h != h1])])
    return S

In [82]:
def specialize_G(x, domains, G, S):
    G_prev = list(G)
    for g in G_prev:
        if g not in G:
            continue
        if fulfills(x, g):
            G.remove(g)
            Gminus = min_specializations(g, domains, x)
            ## keep only specializations that have a conuterpart in S
            G.update([h for h in Gminus if any([more_general(h, s)
                                                for s in S])])
            ## remove hypotheses less general than any other in G
            G.difference_update([h for h in G if 
                                 any([more_general(g1, h) 
                                      for g1 in G if h != g1])])
    return G

In [83]:
candidate_elimination(examples)


 G[0]: {'??????'}

 S[0]: {'000000'}

 G[1]: {'??????'}

 S[1]: {'000000'}

 G[2]: {'??????'}

 S[2]: {'000000'}

 G[3]: {'??????'}

 S[3]: {'000000'}

 G[4]: {'??????'}

 S[4]: {'000000'}

 G[5]: {'??????'}

 S[5]: {'000000'}

 G[6]: {'??????'}

 S[6]: {'000000'}
